In [1]:
# Application of Jarzynski equality to diffusion couple (Fig.7)

In [2]:
import os
import csv
import time
import matplotlib.pyplot as plt
import requests
import numpy as np
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain_core.messages import AIMessage
from scipy.integrate import trapezoid
from scipy.integrate import simpson


In [3]:
#Numerical parameter values
PI = 3.141592 #π
RR = 8.3145 #Gas constant [J／（mol K）]

c0 = 0.5 # alloy comosition（moler fraction of B component）
al = 100.0*1.e-6 #length of 1-dimensional simulation area [m]

#These are not a real time but an increment
real_delt = 0.08 #real time step[s]
itime1 = 0 #Initial time 
itime1max = int(3.*3600./real_delt) # Final time（3hr, Divide by real time step to convert to dimensionless iteration number）
itime1_step = int(10000*0.08/real_delt) #Time step to save the field data
Ndata = int(itime1max/itime1_step+0.5)  #Number of data to plot

D0=8.9e-5; Q0=291000.; #diffusion parameters for the self-diffusion of Fe(fcc)
Mc0=1.0; Mc=Mc0*c0*(1.0-c0) # diffusion Mobility [Dimensionless]

cm_eq=0.07;  cp_eq=0.93;  c_av=0.5*(cm_eq+cp_eq)

#Output data folder
output_folder = "JE_out/"  
os.makedirs(output_folder, exist_ok=True)


In [4]:
# Definition of tool
@tool
# Definition of tool
def Work_of_Diffusion(mesh_number: int, aging_temp: float, interaction_param: float)-> int:
    """
    Performs a diffusion simulation of diffusion couple in A-B binary alloy with regular solution approximation during isothermal aging.
    It receives three input parameters: mesh number, aging temperature, and interaction parameter. 
    The diffusion simulation is performed, and the calculation results are displied on screen amd saved in hard disk. 
    Finally, it returns the end status of the calculation as an int type. When the value of the end status is 0, the calculation fails, 
    and when the value of the end status is 1, the calculation succeeds.

    Args:
        mesh number (int): an integer type input parameter, no unit.
        aging temp (float): temperature, a float type input parameter, unit is K.
        interaction parameter (float): a float type input parameter, unit is J/mol.

    Returns:
        iflg (int): end status of the calculation.
    """

    NDP=mesh_number
    temp=aging_temp
    LL0=interaction_param

    nd=NDP-1
    nd2=int((NDP-1)/2)

    text_nd = str(nd)
    text_temp = str(int(temp))
    text_L0 = str(int(LL0))
    
    fname0 = f"JE_N{text_nd}_T{text_temp}_L{text_L0}"
    fname = f"{fname0}.png"
    path = "./"+output_folder+fname
    

    # Setting constants for calculation conditions
    b1 = al/nd # block size in the difference method [m]
    Rtemp = RR*temp #  RT [J/mol]
    D=D0*np.exp(-Q0/Rtemp) 
    t0=b1*b1/D  # dimensionless time unit 
    delt=real_delt/t0 #time step [dimensionless]
    L0=LL0/Rtemp  # Interaction parameters [J/mol] normalized by RT
    
    # initial composition field
    c = np.concatenate((np.full(nd2, cp_eq), np.full(1, c_av), np.full(nd2, cm_eq)))

    iflg=0

#-------------------------------------------------------------------------------------
    x = al*1.0e+06/nd*(np.arange(0, NDP))
    dG1=np.zeros(Ndata)
    t1=np.zeros(Ndata)
    dW1=np.zeros(Ndata)

    #Free energy at the initial field
    xn=np.linspace(0, nd, NDP) 
    Gc=L0*c*(1.0-c)+c*np.log(c)+(1.-c)*np.log(1.-c)
    #G_av_ini = trapezoid(Gc, xn)/nd
    G_av_ini = simpson(Gc, xn)/nd

    iter=0; dW=0.0
    for itime1 in range(itime1max):
        time1=delt*itime1*t0
        mu=L0*(1.0-2.0*c)+np.log(c)-np.log(1.-c)
        cR=np.roll(c, -1);  cL=np.roll(c, 1) #boundary condition
        cR[nd]=c[nd];  cL[0]=c[0]
        muR=np.roll(mu, -1);  muL=np.roll(mu, 1) #boundary condition件
        muR[nd]=mu[nd];  muL[0]=mu[0]
        mu_dev = muL+muR-2.*mu

        #Diffusion equation
        dcdt = Mc*mu_dev
        c2 = c + dcdt*delt
        res = c2 - (np.mean(c2) - c0)  #Correction for average composition
        c = np.clip(res, 1.0e-06, 1.0-1.0e-06)  #Correction of composition range

        #Work due to diffusion friction
        mu1=L0*(1.0-2.0*c)+np.log(c)-np.log(1.-c)
        mu1R=np.roll(mu1, -1);  mu1L=np.roll(mu1, 1) #boundary condition
        mu1R[nd]=mu1[nd];  mu1L[0]=mu1[0]
        J=-0.5*Mc*(mu1R-mu1L)
        LW=J*J/Mc; ALW=np.exp(-LW); ALW_av = simpson(ALW, xn)/nd;  dW_av=-np.log(ALW_av)
        #dW_av = simpson(LW, xn)/nd
        dW=dW+(-dW_av)*delt
 
        if itime1 % itime1_step == 0:
            Gc=L0*c*(1.0-c)+c*np.log(c)+(1.-c)*np.log(1.-c)
            #G_av = trapezoid(Gc, xn)/nd
            G_av = simpson(Gc, xn)/nd
            dG=G_av-G_av_ini

            t1[iter]=time1
            dG1[iter]=dG*Rtemp
            dW1[iter]=dW*Rtemp        
            print('iter dG dW = ', iter, dG1[iter], dW1[iter])
            iter=iter+1
        
    print('time = ', t1)
    print('dG = ', dG1)
    print('dW = ', dW1)
    iflg=1

    #Drawing the figure
    plt.figure(figsize=(5,5))

    plt.plot(t1, dG1,  marker='o', color="Black")
    plt.plot(t1, dW1,  marker='o', color="Red")
    #plt.plot(x, c, color="Black", label="B")

    plt.xlim(0, 10000)
    plt.ylim(-9000, 0)
    #plt.ylim(-700, 0)
    #plt.ylim(-4000, 0)
    #plt.ylim(-2000, 0)

    plt.xticks(fontsize=Ndata+2)
    plt.yticks(fontsize=Ndata+2)
    plt.grid(True)
    plt.xlabel('Time(s)', fontsize=20)
    plt.ylabel('Free energy change (J/mol)', fontsize=20)
    #plt.show()
    plt.savefig(path, bbox_inches='tight', pad_inches=0.1)
    #plt.pause(5.0) #displayed for 5.0 sec
    time.sleep(5) 
    plt.close()
            
    return iflg

In [5]:
tools = [Work_of_Diffusion]
LLM = "qwen3:8b"
model = ChatOllama( 
    model=LLM, 
    temperature=0 
)
agent = create_agent( 
    model=model, 
    tools=tools 
)


In [6]:
# Run the agent
messages = {
    "messages": [{"role": "user", "content": """Fix the mesh number to 201, the aging temperature at 1600 K, and the interaction parameter to 25000. 
               Under the above conditions, run the tool:'Work_of_Diffusion'."""}]
}
answer = agent.invoke(messages)
messages_list = answer["messages"] # Get the messages list
filtered_messages = [msg.content for msg in messages_list if isinstance(msg, AIMessage)] # Extract assistant messages only.
formatted_output = "\n- " + "\n- ".join(filtered_messages) # Formatted and output
print(formatted_output)


iter dG dW =  0 -0.3713755779281492 -0.1715141904464977
iter dG dW =  1 -173.23185056853936 -167.94785165704872
iter dG dW =  2 -246.58113132460358 -241.2572478511695
iter dG dW =  3 -302.8029942997505 -297.51916028117824
iter dG dW =  4 -350.03401829535954 -344.8508148512801
iter dG dW =  5 -391.34061363640063 -386.2944793878144
iter dG dW =  6 -428.24721927735527 -423.3563959525673
iter dG dW =  7 -461.65182238857426 -456.9232967197423
iter dG dW =  8 -492.1515300951883 -487.58577094014083
iter dG dW =  9 -520.1785009290448 -515.7723295946919
iter dG dW =  10 -546.064030852927 -541.812308427538
iter dG dW =  11 -570.0721646802141 -565.9687787705875
iter dG dW =  12 -592.419071412644 -588.4575130495747
iter dG dW =  13 -613.2851548800543 -609.4588544340584
time =  [    0.   800.  1600.  2400.  3200.  4000.  4800.  5600.  6400.  7200.
  8000.  8800.  9600. 10400.]
dG =  [-3.71375578e-01 -1.73231851e+02 -2.46581131e+02 -3.02802994e+02
 -3.50034018e+02 -3.91340614e+02 -4.28247219e+02 -4.

In [7]:
# Run the agent
messages = {
    "messages": [{"role": "user", "content": """Fix the aging temperature to 1600 K and the interaction parameter to -25000. Under these conditions, 
    run the “Work_of_Diffusion” tool repeatedly with the  mesh number ranging from 101 to 601 in increments of 100."""}]
}
answer = agent.invoke(messages)
messages_list = answer["messages"] # Get the messages list
filtered_messages = [msg.content for msg in messages_list if isinstance(msg, AIMessage)] # Extract assistant messages only.
formatted_output = "\n- " + "\n- ".join(filtered_messages) # Formatted and output
print(formatted_output)


iter dG dW =  0 -3.508809519972776 -0.7401653244661668
iter dG dW =  0 -6.9081635599690925 -1.470984605137264
iter dG dW =  0 -10.103932157692874 -2.2050340058228164
iter dG dW =  0 -13.024597222039619 -2.9492569883944775
iter dG dW =  0 -15.619717341883062 -3.710989708080766
iter dG dW =  0 -17.8540006506262 -4.49374294234313
iter dG dW =  1 -2437.6998387757885 -2174.7968441795438
iter dG dW =  1 -2471.583613110099 -2332.2957572843934
iter dG dW =  1 -2483.0355894827994 -2387.5679830700597
iter dG dW =  1 -2488.7910282275657 -2415.58320336426
iter dG dW = iter dG dW =  2 -3507.990725023687 -3368.6066010084974
 2 -3472.8000046947895 -3207.3981645786093
iter dG dW =  2 -3525.7934876586805 -3452.81816119417
iter dG dW =  2 -3519.843730158164 -3424.5642716166094
iter dG dW =  3 -4259.844032163513 -3997.4482692967595
iter dG dW =  3 -4297.339592276829 -4159.945072439988
iter dG dW =  3 -4316.286533355659 -4244.423618397547
iter dG dW =  3 -4309.955943021154 -4216.108788592537
iter dG dW = 

In [9]:
# Run the agent
messages = {
    "messages": [{"role": "user", "content": """Fix the mesh number to 401 and the aging temperature to 1600 K. Under these conditions, 
    run the “Work_of_Diffusion” tool repeatedly with the interaction parameter ranging from -25000 to 25000 in increments of 5000."""}]
}
answer = agent.invoke(messages)
messages_list = answer["messages"] # Get the messages list
filtered_messages = [msg.content for msg in messages_list if isinstance(msg, AIMessage)] # Extract assistant messages only.
formatted_output = "\n- " + "\n- ".join(filtered_messages) # Formatted and output
print(formatted_output)


iter dG dW =  0 -13.024597222039619 -2.9492569883944775
iter dG dW =  0 -11.13208757806968 -2.73456818168538
iter dG dW =  0 -9.38266985072641 -2.498989412290476
iter dG dW =  0 -7.777837560534159 -2.2436548670663083
iter dG dW =  0 -6.319072193323631 -1.9708315302649686
iter dG dW =  0 -5.00784210884124 -1.684278391190963
iter dG dW =  0 -3.845601312965646 -1.3896518510987268
iter dG dW =  0 -2.8337880715932013 -1.0948650177494517
iter dG dW =  0 -1.9738233398095193 -0.8102521287837668
iter dG dW =  0 -1.2671089744974278 -0.5483589677695997
iter dG dW =  0 -0.7150256917682569 -0.3232062740120099
iter dG dW =  1 -2488.7910282275657 -2415.58320336426
iter dG dW =  1 -1612.7917931666902 -1572.4433880647766
iter dG dW =  1 -2182.7433266957687 -2121.7358538001176
iter dG dW = iter dG dW =  1 -875.6109650826954 -857.3089303554108
 1 -312.0301675336743 -306.50603342611953
iter dG dW =  1 -1350.3891230013348 -1318.5418763021698
iter dG dW =  1 -665.901342667356 -652.7719648550441
iter dG dW =